# TransLuxPop — Transportation Feature Extraction Pipeline (OSMnx)
This notebook is a cleaned and reproducible version of `transportation_automation.ipynb`:
- Deduplicated imports and settings
- Centralized configuration (paths, Overpass endpoints, parameters)
- Refactored core logic into reusable functions
- Added checkpointing (resume) and failure retries

> Tip: export these functions into scripts under `code/` for a fully reproducible GitHub release.


## 0. Environment & Dependencies
建议版本：
- osmnx >= 1.8（或 2.x，代码做兼容）
- geopandas, shapely, networkx
- pandas, numpy, tqdm


In [ ]:
# ===== Imports =====
import os
import time
import math
import numpy as np
import pandas as pd
from tqdm import tqdm

import osmnx as ox
import networkx as nx

# Optional: silence warnings
import warnings
warnings.filterwarnings("ignore")


## 1. Global Configuration (edit here only)

In [ ]:
# ===== Config =====
OVERPASS_URLS = [
    "https://overpass.kumi.systems/api/interpreter",
    # 备用源（可按需要添加）
    # "https://overpass-api.de/api/interpreter",
]

# 输入：你的 grid 列表（Excel/CSV）
GRID_XLSX_PATH = "PATH/TO/grids_set_3_HQ.xlsx"

# 输出：写回的 dataset（建议写到本地 data/ 目录）
OUTPUT_XLSX_PATH = "PATH/TO/grids_set_3_with_trans.xlsx"

# checkpoint：每 N 个 grid 保存一次，防止中途崩掉
CHECKPOINT_EVERY = 10

# OSMnx 设置
ox.settings.use_cache = True
ox.settings.log_console = False


## 2. Utilities: OSMnx compatibility + retries + projected length

In [ ]:
def set_overpass(url: str):
    ox.settings.overpass_url = url

def _graph_from_bbox_compat(north, south, east, west, **kwargs):
    """兼容 osmnx 1.x / 2.x 的 bbox 抓取接口"""
    try:
        # osmnx 2.x
        return ox.graph.graph_from_bbox(north, south, east, west, **kwargs)
    except Exception:
        # osmnx 1.x
        return ox.graph_from_bbox(north=north, south=south, east=east, west=west, **kwargs)

def fetch_graph_with_retry(north, south, east, west, network_type="drive", simplify=True,
                           truncate_by_edge=True, max_retry=3, sleep_s=2):
    """抓取 OSM graph：自动切 overpass + 重试"""
    last_err = None
    for attempt in range(max_retry):
        url = OVERPASS_URLS[attempt % len(OVERPASS_URLS)]
        set_overpass(url)
        try:
            G = _graph_from_bbox_compat(
                north, south, east, west,
                network_type=network_type,
                simplify=simplify,
                truncate_by_edge=truncate_by_edge
            )
            return G
        except Exception as e:
            last_err = e
            time.sleep(sleep_s * (attempt + 1))
    raise last_err

def project_and_get_edges(G):
    """投影到米制坐标并返回 edges GeoDataFrame（用于精确长度）"""
    Gp = ox.project_graph(G)
    edges = ox.graph_to_gdfs(Gp, nodes=False, edges=True, fill_edge_geometry=True)
    if "geometry" in edges.columns:
        edges["length_m"] = edges.geometry.length
    else:
        # fallback
        edges["length_m"] = edges.get("length", np.nan)
    return edges

def highway_len_km(edges, hw_types):
    """统计指定 highway 类型的长度（km）"""
    if edges.empty:
        return 0.0
    if "highway" not in edges.columns:
        return 0.0
    # highway 字段有时是 list
    hw = edges["highway"].apply(lambda x: x[0] if isinstance(x, list) and len(x) else x)
    mask = hw.isin(hw_types)
    return float(edges.loc[mask, "length_m"].sum() / 1000.0)

def compute_intersections(G):
    """用 consolidate_intersections 统计交叉口数量（更稳）"""
    # consolidate 需要有几何
    Gp = ox.project_graph(G)
    # 注意：consolidate_intersections 对 directed graph 更友好
    Gc = ox.simplification.consolidate_intersections(Gp, tolerance=15, rebuild_graph=True, dead_ends=False)
    return int(len(Gc.nodes))


## 3. Per-grid transportation feature computation (core)

In [ ]:
# 你可按论文/数据集定义调整 highway 分类
HW_MOT = ["motorway"]
HW_TRU = ["trunk"]
HW_PRI = ["primary"]
HW_SEC = ["secondary"]
HW_TER = ["tertiary"]

# 城市道路（你定义的 urban）
HW_URB = ["residential", "unclassified", "service", "living_street"]

def compute_transport_features(lat_min, lon_min, lat_max, lon_max):
    """返回一个 dict：len_mot_km, len_pri_km, ..., intersec_count 等"""
    north, south, east, west = lat_max, lat_min, lon_max, lon_min

    G = fetch_graph_with_retry(north, south, east, west)
    if G is None or len(G.nodes) == 0:
        return {
            "intersec": 0,
            "len_mot_km": 0.0, "len_tru_km": 0.0, "len_pri_km": 0.0,
            "len_sec_km": 0.0, "len_ter_km": 0.0, "len_urb_km": 0.0
        }

    edges = project_and_get_edges(G)

    feat = {}
    feat["intersec"] = compute_intersections(G)
    feat["len_mot_km"] = highway_len_km(edges, HW_MOT)
    feat["len_tru_km"] = highway_len_km(edges, HW_TRU)
    feat["len_pri_km"] = highway_len_km(edges, HW_PRI)
    feat["len_sec_km"] = highway_len_km(edges, HW_SEC)
    feat["len_ter_km"] = highway_len_km(edges, HW_TER)
    feat["len_urb_km"] = highway_len_km(edges, HW_URB)
    return feat


## 4. Batch processing (with checkpoint & resume)

In [ ]:
df = pd.read_excel(GRID_XLSX_PATH)

# 你需要保证表中有这些列名（必要时在这里 rename）
required_cols = ["lat_min", "lon_min", "lat_max", "lon_max"]
for c in required_cols:
    if c not in df.columns:
        raise ValueError(f"Missing column: {c}")

# 初始化输出列（若不存在）
out_cols = ["intersec", "len_mot_km", "len_tru_km", "len_pri_km", "len_sec_km", "len_ter_km", "len_urb_km"]
for c in out_cols:
    if c not in df.columns:
        df[c] = np.nan

# 如果要断点续跑：只跑缺失的行
todo_idx = df.index[df["intersec"].isna()].tolist()
print(f"Total rows: {len(df)}, to compute: {len(todo_idx)}")


In [ ]:
for j, idx in enumerate(tqdm(todo_idx)):
    row = df.loc[idx]
    try:
        feat = compute_transport_features(row.lat_min, row.lon_min, row.lat_max, row.lon_max)
        for k, v in feat.items():
            # 对齐列名
            if k == "intersec":
                df.at[idx, "intersec"] = v
            else:
                df.at[idx, k] = v
    except Exception as e:
        # 标记失败（也可以写入 error log）
        df.at[idx, "intersec"] = -1
        # 你也可以把 error 信息写到单独一列
        # df.at[idx, "error_msg"] = str(e)[:200]

    if (j + 1) % CHECKPOINT_EVERY == 0:
        df.to_excel(OUTPUT_XLSX_PATH, index=False)

# 最终保存
df.to_excel(OUTPUT_XLSX_PATH, index=False)
print("Saved:", OUTPUT_XLSX_PATH)


## 5. Optional: generate normalized *_use features (for cross-country generalization)
Your table already uses the *_use idea; here is a standard approach:
- Normalize by cell_area (or bbox area)
- Or normalize by total road length


In [ ]:

if "cell_area_km2" in df.columns:
    eps = 1e-6
    df["Intersec_use"] = df["intersec"] / (df["cell_area_km2"] + eps)
    for hw in ["mot","tru","pri","sec","ter","urb"]:
        df[f"{hw}_use"] = df[f"len_{hw}_km"] / (df["cell_area_km2"] + eps)

    df.to_excel(OUTPUT_XLSX_PATH, index=False)
